We are doing several unconventional things here. We are hooking up Jupyter and Twisted so that the so-called "Python kernel" will be able to run at the same time with the Twisted event loop. We are also running Klein in a way that allows us to interactively add to the route table. For both of those reasons, we need to do some "magical set-up". Note that for running Klein outside of a Jupyter notebook, with the usual workflow for developing a web-app (i.e., "run, CTRL-C, make changes, run"), this prelude is replaced by

    from klein import run, route
    @route('/')
    def home(request):
        return 'Hello, world!'
    run("localhost", 8080)

In [1]:
from __future__ import print_function
from tornado.platform.twisted import install
install()
from twisted.internet import reactor
from klein import route, resource
from twisted.web.server import Site
from twisted.internet.endpoints import TCP4ServerEndpoint
from twisted.internet import defer
from twisted.python import log

lport, endpoint = [], TCP4ServerEndpoint(reactor, 8080)
def listen():
    if lport:
        port = lport.pop()
        d = port.stopListening()
    else:
        d = defer.succeed(None)
    def actuallyListen(dummy):
        s = Site(resource())
        return endpoint.listen(Site(resource()))
    def finishedListening(port):
        lport.append(port)
        print("Ready to serve web requests on {}".format(port))
    d.addCallback(actuallyListen)
    d.addCallback(finishedListening)

In [2]:
@route('/')
def home(request):
    return 'Hello, world!'
listen()

Ready to serve web requests on <<class 'twisted.internet.tcp.Port'> of twisted.web.server.Site on 8080>


In [3]:
@route('/lala')
def thingy(request):
    return 'lolo'
listen()

Ready to serve web requests on <<class 'twisted.internet.tcp.Port'> of twisted.web.server.Site on 8080>
